# Images to label #
Demonstration of GA ability to generate adversarial pattern for misclassifying a set of images to given label.

In [ ]:
%cd ..

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from evgena.dataset import Dataset
from evgena.genetals import Images2LabelObjectiveFnc
from evgena.models import TfModel
from evgena.metrics import mse
from evgena.utils.large_files import maybe_download
from genetals.core import *
from genetals.initializers import MultivariateRandomInit
from genetals.operators import ShuffleOperator, TwoPointXover, BiasedMutation, NSGAOperator
from genetals.callbacks import MultiObjectiveReport, GAStatus

Model to be "attacked"

In [ ]:
model = TfModel(
    'models/fashion_mnist_cnn/model',
    'end_points/images',
    'end_points/scores',
    batch_size=8192
)

Set of images to convert

In [ ]:
fashion_mnist = Dataset.from_nprecord(maybe_download('datasets/fashion_mnist.npz'))

source_class = 0
target_class = 5
images = Dataset.sub_dataset(fashion_mnist, [source_class], do_stratified=False).train.X

Visualizations

In [ ]:
%matplotlib ipympl

fig, ax = plt.subplots(1, 1, figsize=(9,4))
# fig.tight_layout()

ax.set_xlim(0.0000000001, 1)
ax.set_xlabel('Target class prediction probability')
ax.set_ylim(-0.4, 0)
ax.set_ylabel('similarity')
ax.set_xscale('log')
ax.grid(axis='both')
ax.vlines(0.5, -1, 1, colors='g')

GA definition

In [ ]:
initializer = MultivariateRandomInit.uniform(
    (14, 14),
    scales=np.random.permutation((np.exp(np.linspace(3, 5, 100)) - 1) / (np.exp(5) - 1))
)

graph = OperatorGraph()
select_op = ShuffleOperator(graph.init_op)
xover_op = TwoPointXover(select_op, 0.6)
mutation_op = BiasedMutation(xover_op, sigma=0.1, l_bound=-1, u_bound=1)
moea_op = NSGAOperator(graph.init_op, mutation_op)

callbacks = [MultiObjectiveReport(ax), GAStatus(fig)]

ga = GeneticAlgorithm(
    initializer=initializer,
    operator_graph=graph,
    objective_fnc=Images2LabelObjectiveFnc(
        model, lambda x, y: - mse(x, y),
        target_class, images,
        sample_size=64, sample_ttl=0.98
    ),
    callbacks=callbacks,
    results_dir='logs/images2label'
)

GA run

In [ ]:
%time final_pop, fitnesses, objectives = ga.run(population_size=256, generation_cap=64)